In [1]:
# Import packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Import the DEA data and export dataset with non-zero/non-blank observations
df1 = pd.read_excel('Compustat DEA.xlsx')

In [ ]:
df1.columns

In [ ]:
# Just the important columns
df1 = df1[['Global Company Key','Data Year - Fiscal','Assets - Total','Employees','Revenue - Total','Standard Industry Classification Code','Research and Development Expense']]

In [ ]:
# Created lagged R&D expense
df1['lagged_rnd'] = df1.sort_values(by=['Global Company Key','Data Year - Fiscal']).groupby(['Global Company Key'])['Research and Development Expense'].shift(1)

In [ ]:
# Filter the dataframe for only the columns with non negative values
df1 = df1[(df1['Assets - Total']>0) & (df1['Employees']>0) & (df1['Revenue - Total']>0) & (df1['lagged_rnd']>0)]

In [ ]:
df1.to_excel('Compustat_DEA_v3.xlsx')

In [2]:
# Import the data
df = pd.read_excel('Compustat_DEA_v3.xlsx')

In [3]:
# List of years
year = [1996, 1997, 1998, 1999, 2000, 2001,2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [40]:
df_year = df[df['Year']==2017]
md = smf.mixedlm("ln_revenue ~ ln_assets + ln_comp + ln_patent + ln_emp", df_year, groups=df_year["GVKEY"], re_formula = "ln_assets + ln_comp + ln_patent + ln_emp")
mdf = md.fit()

C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [41]:
print(mdf.summary())

                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     ln_revenue
No. Observations:      969         Method:                 REML      
No. Groups:            969         Scale:                  0.0169    
Min. group size:       1           Likelihood:             -949.7555 
Max. group size:       1           Converged:              Yes       
Mean group size:       1.0                                           
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                  0.488    0.084   5.786 0.000  0.322  0.653
ln_assets                  0.865    0.019  45.577 0.000  0.828  0.903
ln_comp                   -0.093    0.008 -11.751 0.000 -0.108 -0.077
ln_patent                  0.012    0.018   0.655 0.512 -0.024  0.048
ln_emp                     0.333    

C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\base\model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [4]:
df.shape

(97744, 17)

In [5]:
df = df[df['ln_patent'].notna()]

In [6]:
df.shape

(37772, 17)

In [104]:
year1 = [2017]

In [105]:
df_final = [] # Store the RE coefficients for all groups
#df_final_fe = {'year':[],'intercept':[],'ln_assets':[],'ln_comp':[],'ln_patent':[],'ln_emp':[] } # Store the FE coefficients of all years

for y in year1:
    df_year = df[df['Year']==y]
    md = smf.mixedlm("ln_revenue ~ ln_assets + ln_comp + ln_patent + ln_emp", df_year, groups=df_year["GVKEY"], re_formula = "ln_assets + ln_comp + ln_patent + ln_emp")
    mdf = md.fit()
    
    
    dictionary = mdf.random_effects
    df_coeff = pd.DataFrame.from_dict(dictionary, orient='index')
    df_coeff['year'] = y
    df_final.append(df_coeff)
    
    '''
    dictionary1 = mdf.fe_params
    df_final_fe['year'].append(y)
    df_final_fe['intercept'].append(dictionary1[0])
    df_final_fe['ln_assets'].append(dictionary1[1])
    df_final_fe['ln_comp'].append(dictionary1[2])
    df_final_fe['ln_patent'].append(dictionary1[3])
    df_final_fe['ln_emp'].append(dictionary1[4])
    
    '''
    
    
    

C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2059: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  ConvergenceWarning)
C:\Users\Devashish\Anaconda3_1\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [106]:
#fe_estimates = pd.DataFrame.from_dict(df_final_fe)

re_estimates = pd.DataFrame()

for d in df_final:
    df_re = pd.DataFrame(d,columns=['Group','ln_assets','ln_comp','ln_patent','ln_emp','year'])
    re_estimates = re_estimates.append(df_re)

#all_years = pd.merge(re_estimates, fe_estimates, left_on = 'year', right_on='year', how='left')

In [107]:
re_estimates.to_excel('re_estimates_patents2017.xlsx')

In [ ]:
#fe_estimates.to_excel('fe_estimates._patents.xlsx')

In [ ]:
all_years.to_excel('all_years_patents.xlsx')